In [ ]:
from pathlib import Path
import sys

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from topostats.io import LoadScans

src_dir = Path("../src")
sys.path.append(str(src_dir))
from unet import dice_loss, iou_loss

In [ ]:
model_path = Path("/Users/sylvi/topo_data/picoz/20250704-picoz-vheight-augment.keras")
custom_objects = {"dice_loss": dice_loss, "iou_loss": iou_loss}
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

In [ ]:
# image_file = Path("/Users/sylvi/topo_data/picoz/output_abs_thresh/20250528_RA_dose_0GY_picoz/processed/20250528_picoz_0RA_tip_0.0_00025.topostats")
image_file = Path("/Users/sylvi/topo_data/picoz/output_unet/20250528_RA_dose_0GY_picoz/processed/20250528_picoz_0RA_tip_0.0_00014.topostats")
assert image_file.exists(), f"Image file {image_file} does not exist."

loadscans = LoadScans([image_file], channel="dummy")
loadscans.get_data()
img_dict = loadscans.img_dict
print(img_dict.keys())
image_data = img_dict[list(img_dict.keys())[0]]
print(image_data.keys())
image = image_data["image"]
plt.imshow(image)
plt.show()

image = np.expand_dims(image, axis=0)  # Add batch dimension
image = np.expand_dims(image, axis=-1)  # Add channel dimension
print(image.shape)
predicted_mask = model.predict(image)

plt.imshow(predicted_mask[0, :, :, 0], cmap='gray')
plt.title("Predicted Mask")
plt.show()

predicted_mask_binary = (predicted_mask[0, :, :, 0] > 0.5).astype(np.uint8)
plt.imshow(predicted_mask_binary, cmap='gray')
plt.title("Predicted Mask (Binary)")
plt.show()